<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BIOASQ_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os

print(os.listdir("/content/drive/MyDrive/Thesis_B"))

['rouge_8b_mod.csv', '.ipynb_checkpoints', 'test_data.csv', 'train_data.csv']


In [2]:
DATA_DIR = "/content/drive/MyDrive/Thesis_B"

file = pd.read_csv(os.path.join(DATA_DIR,'rouge_8b_mod.csv'))
f_values = file.values
# print(f_values[:10])
# print(col_names[:10])
q_id_split = np.split(f_values, np.where(np.diff(f_values[:,0]))[0]+1)
# print(q_id_split[:2])

In [3]:
train_data_size = round(len(q_id_split)*0.8)
test_data_size = round(len(q_id_split) - train_data_size)
print(train_data_size)
print(test_data_size)

2594
648


In [6]:
import random
random.seed(3007)
random.shuffle(q_id_split)
print(q_id_split[:1])

[array([[984, '5547a7e9f35db75526000006', 4, 0.36687, 0.31138000000000005,
        0.29091, 0.26994, 0.30142,
        'During nucleotide excision repair (NER) in bacteria the UvrC nuclease and the short oligonucleotide that contains the DNA lesion are removed from the post-incision complex by UvrD, a superfamily 1A helicase.',
        1],
       [984, '5547a7e9f35db75526000006', 17, 0.2436, 0.23376, 0.22369,
        0.21333, 0.21682,
        'Damage recognition during bacterial NER depends upon UvrA, which binds to the damage and loads UvrB onto the DNA.',
        1],
       [984, '5547a7e9f35db75526000006', 3, 0.2436, 0.23376, 0.22369,
        0.21333, 0.21682,
        'In eubacteria, the NER system typically consists of UvrA, UvrB, UvrC, the UvrD helicase, DNA polymerase I, and ligase.',
        1],
       [984, '5547a7e9f35db75526000006', 6, 0.3237, 0.21052, 0.16568,
        0.14371, 0.19881,
        'PcrA helicase from Bacillus stearothermophilus can also displace UvrC and the exci

In [7]:
train_data = q_id_split[:2594]
test_data = q_id_split[2594:]
f_values_train = np.concatenate(train_data, axis=0)
f_values_test = np.concatenate(test_data, axis=0)
col_names = file.columns
print(col_names)

train_df = pd.DataFrame(f_values_train, columns = col_names)
test_df = pd.DataFrame(f_values_test, columns = col_names)

Index(['qid', 'pubmedid', 'sentid', 'N1', 'N2', 'L', 'S4', 'SU4',
       'sentence text', 'SU4_labels'],
      dtype='object')


In [8]:
print(train_data[:3])
print('**************8')
print(test_data[:3])

[array([[984, '5547a7e9f35db75526000006', 4, 0.36687, 0.31138000000000005,
        0.29091, 0.26994, 0.30142,
        'During nucleotide excision repair (NER) in bacteria the UvrC nuclease and the short oligonucleotide that contains the DNA lesion are removed from the post-incision complex by UvrD, a superfamily 1A helicase.',
        1],
       [984, '5547a7e9f35db75526000006', 17, 0.2436, 0.23376, 0.22369,
        0.21333, 0.21682,
        'Damage recognition during bacterial NER depends upon UvrA, which binds to the damage and loads UvrB onto the DNA.',
        1],
       [984, '5547a7e9f35db75526000006', 3, 0.2436, 0.23376, 0.22369,
        0.21333, 0.21682,
        'In eubacteria, the NER system typically consists of UvrA, UvrB, UvrC, the UvrD helicase, DNA polymerase I, and ligase.',
        1],
       [984, '5547a7e9f35db75526000006', 6, 0.3237, 0.21052, 0.16568,
        0.14371, 0.19881,
        'PcrA helicase from Bacillus stearothermophilus can also displace UvrC and the exci

In [9]:
# print(train_df.head())
# print(test_df.head())

print(type(train_df))
print(type(test_df))
print(train_df.columns.values)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels']


In [10]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [12]:
def data_grouplists(data):
  data_list = data.groupby('qid', as_index=False)['SU4_labels','sentence text'].agg(list)
  return data_list

In [13]:
train_list = data_grouplists(train_df)
test_list = data_grouplists(test_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [15]:
def add_specialtokens(data_list):
  specialtok_sentences = []

  for sentence in data_list['sentence text']:
    each_specialtoken = []
    for each in sentence:
      each_specialtoken.append(["[CLS] " + each + " [SEP]" ])
    specialtok_sentences.append(each_specialtoken)
  return specialtok_sentences

In [16]:
train_specialtok = add_specialtokens(train_list)
test_specialtok = add_specialtokens(test_list)

In [17]:
print(train_specialtok[0])
print(train_specialtok[1])

[['[CLS] the 7 known EGFR ligands (EGF, betacellulin, epiregulin, heparin-binding EGF, transforming growth factor-α [TGF-α], amphiregulin, and epigen) [SEP]'], ['[CLS]  EGFR ligands epidermal growth factor (EGF), amphiregulin (AREG) and transforming growth factor alpha (TGFα) [SEP]'], ['[CLS] mammalian EGFR ligands including EGF, TGF-α (TGFα), amphiregulin (AREG), heparin-binding EGF-like growth factor (HB-EGF), betacellulin, epiregulin, and epigen. [SEP]'], ['[CLS] the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG) [SEP]'], ['[CLS] Among EGFR ligands, heparin-binding epidermal growth factor (HB-EGF) [SEP]'], ['[CLS] Among EGFR ligands, heparin-binding EGF-like growth factor, TGF-α and Betacellulin (BTC) are produced in the tumor microenvironment of FDC-S at RNA level. [SEP]'], ['[CLS] oluble amphiregulin (AR), transforming growth factor alpha (TGFα), neuregulin 2 beta, and epigen stimulate greater EGFR coupling to cell pr

In [18]:
def tokenize_sentences(specialtok_sentences):
  tokenized_sentences = []

  for sentence in specialtok_sentences:
    eachsent_tokenized = []
    for each in sentence:
      eachsent_tokenized.append(tokenizer.tokenize(each[0]))
    tokenized_sentences.append(eachsent_tokenized)
  return tokenized_sentences

In [19]:
train_tokenized = tokenize_sentences(train_specialtok)
test_tokenized = tokenize_sentences(test_specialtok)


In [20]:
def data_tokenizedsent(tokenized_sentences, data_list):
  data_list['tokenized_sentences'] = tokenized_sentences
  return data_list

In [21]:
print(train_tokenized[0])

[['[CLS]', 'the', '7', 'known', 'e', '##gf', '##r', 'ligand', '##s', '(', 'e', '##gf', ',', 'beta', '##cel', '##lu', '##lin', ',', 'ep', '##ire', '##gul', '##in', ',', 'he', '##par', '##in', '-', 'binding', 'e', '##gf', ',', 'transforming', 'growth', 'factor', '-', 'α', '[', 't', '##gf', '-', 'α', ']', ',', 'amp', '##hire', '##gul', '##in', ',', 'and', 'ep', '##igen', ')', '[SEP]'], ['[CLS]', 'e', '##gf', '##r', 'ligand', '##s', 'ep', '##ider', '##mal', 'growth', 'factor', '(', 'e', '##gf', ')', ',', 'amp', '##hire', '##gul', '##in', '(', 'are', '##g', ')', 'and', 'transforming', 'growth', 'factor', 'alpha', '(', 't', '##gf', '##α', ')', '[SEP]'], ['[CLS]', 'mammalian', 'e', '##gf', '##r', 'ligand', '##s', 'including', 'e', '##gf', ',', 't', '##gf', '-', 'α', '(', 't', '##gf', '##α', ')', ',', 'amp', '##hire', '##gul', '##in', '(', 'are', '##g', ')', ',', 'he', '##par', '##in', '-', 'binding', 'e', '##gf', '-', 'like', 'growth', 'factor', '(', 'h', '##b', '-', 'e', '##gf', ')', ',', 'b

In [22]:
data_train_tokenized = data_tokenizedsent(train_tokenized, train_list)
data_test_tokenized = data_tokenizedsent(test_tokenized, test_list)

In [23]:
print(len(train_tokenized))
print(data_train_tokenized.columns.values)

2594
['qid' 'SU4_labels' 'sentence text' 'tokenized_sentences']


In [24]:
print(len(train_tokenized[0]))
print(len(train_tokenized[1]))
print(len(train_tokenized[2]))

16
11
7


In [25]:
def token2ids(tokenized_sentences):
  # Set the maximum sequence length. 
  MAX_LEN = 512
  token_ids = []

  for tokenized_sentence in tokenized_sentences:
    tokens_to_ids = [tokenizer.convert_tokens_to_ids(each) for each in tokenized_sentence]
    tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    token_ids.append(tokens_to_ids)
  return token_ids

In [26]:
train_tokens2ids = token2ids(train_tokenized)
test_tokens2ids = token2ids(test_tokenized)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors


In [29]:
print(len(train_tokens2ids))
print(len(train_tokens2ids[0]))
print(len(train_tokens2ids[1]))

2594
16
11


In [30]:
def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    eachtid_mask = []
    for each in tid:
      mask = [float(i>0) for i in each] #for each token 
      eachtid_mask.append(mask)#for each sentence
    attention_masks.append(eachtid_mask)
  return attention_masks

In [31]:
train_masks = create_masks(train_tokens2ids)
test_masks = create_masks(test_tokens2ids)

In [32]:
print(len(train_masks))
print(len(train_masks[0]))
print(len(train_masks[0][0]))
print(len(train_masks[1]))
print(len(train_masks[1][1]))


2594
16
512
11
512
